In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)

In [3]:
resultadoAmsa = sf.query_all(
        """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
        
        
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
        
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

,Año,Mes,Cantidad
0,2023,Enero,4091
1,2023,Febrero,267
2,2023,Marzo,195
3,2023,Mayo,3150
4,2023,Junio,346
5,2023,Julio,125
6,2023,Agosto,1
7,2023,Septiembre,174
8,2023,Octubre,62
9,2022,Noviembre,3


In [4]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape


(7762, 8)

In [5]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(7452, 5)

In [6]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(7768, 10)

In [7]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [8]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape


(1821700, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [9]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape


(1436465, 4)

In [10]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1436465, 13)

In [11]:
empresas_unidas.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [ ]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [ ]:
# pacientes_empresas22.shape

In [ ]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [ ]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [12]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Vida Security"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [13]:
Pacientes_Archivo.head(10)


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email
0,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,7,10315646-7,TITULAR,VIRGILIO,GALLARDO VALLADARES,1974-06-07 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,VIRGILIO_GALLARDO_V@HOTMAIL.COM
1,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,5,10375797-5,TITULAR,PEDRO,SANHUEZA SALAZAR,1966-10-15 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,PEDROFEDERICO@GMAIL.COM
2,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,7,8893633-7,CÓNYUGE,BERNARDITA,ALTAMIRANO GOMEZ,1963-02-12 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN
3,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,7,10466504-7,TITULAR,RODRIGO,GAJARDO CARRASCO,1972-09-01 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,RODRIGOAGAJARDOC@GMAIL.COM
4,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,2,11962025-2,CÓNYUGE,MONICA ADRIANA,JARA ROMERO,1972-05-04 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN
5,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,7,20903987-7,HIJO,TOMAS ARTURO,GAJARDO JARA,2002-08-22 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN
6,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,1,10484487-1,TITULAR,LUÍS,MUÑOZ ARAYA,1967-12-25 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,LUIS.MUNOZGB67@GMAIL.COM
7,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,6,12005180-6,CÓNYUGE,MARGARITA,GUAJARDO FERNANDEZ,1974-02-18 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN
8,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,K,19796765-K,HIJO,CAMILA,MUNOZ GUAJARDO,1997-11-28 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN
9,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,5,10489427-5,TITULAR,JOSÉ,SÁNCHEZ INZUNZA,1965-07-04 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,JOSE_.SANCHEZ@HOTMAIL.COM


In [14]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [15]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10315646-7,TITULAR,VIRGILIO,GALLARDO VALLADARES,1974-06-07 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,VIRGILIO_GALLARDO_V@HOTMAIL.COM,103156467
1,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10375797-5,TITULAR,PEDRO,SANHUEZA SALAZAR,1966-10-15 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,PEDROFEDERICO@GMAIL.COM,103757975
2,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,8893633-7,CÓNYUGE,BERNARDITA,ALTAMIRANO GOMEZ,1963-02-12 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN,88936337
3,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10466504-7,TITULAR,RODRIGO,GAJARDO CARRASCO,1972-09-01 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,RODRIGOAGAJARDOC@GMAIL.COM,104665047
4,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,11962025-2,CÓNYUGE,MONICA ADRIANA,JARA ROMERO,1972-05-04 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN,119620252


In [16]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13448\3578335771.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10315646-7,TITULAR,VIRGILIO,GALLARDO VALLADARES,1974-06-07 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,VIRGILIO_GALLARDO_V@HOTMAIL.COM,103156467
1,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10375797-5,TITULAR,PEDRO,SANHUEZA SALAZAR,1966-10-15 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,PEDROFEDERICO@GMAIL.COM,103757975
2,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,8893633-7,CÓNYUGE,BERNARDITA,ALTAMIRANO GOMEZ,1963-02-12 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN,88936337
3,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,10466504-7,TITULAR,RODRIGO,GAJARDO CARRASCO,1972-09-01 00:00:00,M,2023-02-01 00:00:00,2024-01-31 00:00:00,RODRIGOAGAJARDOC@GMAIL.COM,104665047
4,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,11962025-2,CÓNYUGE,MONICA ADRIANA,JARA ROMERO,1972-05-04 00:00:00,F,2023-02-01 00:00:00,2024-01-31 00:00:00,NaN,119620252


In [17]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13448\2830316430.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13448\2830316430.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [18]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(31349, 36)

In [19]:
Pacientes_con_id.head()


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,True,0018c00002bIANTAA4,103156467,Virgilio Gallardo Valladares,RUT,None,None,None,Hombre,None
1,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,True,0018c00002bKNnfAAG,103757975,Pedro Sanhueza Salazar,RUT,None,None,None,Hombre,None
2,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,True,0018c00002bJwB8AAK,88936337,Bernardita Altamirano Gomez,RUT,None,None,None,Mujer,None
3,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,True,0018c00002bHqx9AAC,104665047,Rodrigo Alexis Gajardo Carrasco,RUT,None,None,None,Hombre,None
4,96,CARE ASSISTANCE Plan BASICO,281608,77894990,3,ORAFTI CHILE S.A.,77894990,3,ORAFTI CHILE S.A.,33688,...,True,0018c00002bKBlRAAW,119620252,Monica Adriana Jara Romero,RUT,None,None,None,Mujer,None


In [20]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [21]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (Pacientes_con_id["email"].astype(str).str.lower())
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(lambda y: isValid(y))
    #Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    #Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
476,False,Gaspar Moises,Pardo Castillo,281624458,,2023-05-03 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
571,False,Sergio Gerardo,Olivares De La Barra,92108384,sergio.olivares@orafti.cl,1961-10-03 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
572,False,Gerardo,Olivares Cortazar,202679137,,2000-01-21 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
573,False,Benjamin,Olivares Cortazar,20976024K,,2002-02-22 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
600,False,Edmundo,Hidalgo Becerra,101625397,,1967-10-18 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [22]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [23]:
""" print(dia_hoy) """


' print(dia_hoy) '

In [24]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
385548,a1Y8c00000CldqXEAR,0018c00002azlRDAAY,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
386396,a1Y8c00000CleLsEAJ,0018c00002azmGuAAI,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
449184,a1Y8c00000Cm2JJEAZ,0018c00002b0RUiAAM,a1W8c0000083rZsEAI,Vida Security,0018c00002axQfOAAU,Minera Escondida Ltda.,Grupo BHP,Vida Security,Minera Escondida Ltda.,282349,79587210-8,EMP6151,A-9244
465143,a1Y8c00000CmDFMEA3,0018c00002bHw4eAAC,a1W8c0000083rZiEAI,Vida Security,0018c00002axQfKAAU,Empresa De Transporte Ferroviario S.A.,Ferronor,Vida Security,Empresa De Transporte Ferroviario S.A.,284254,96545600-7,EMP6147,A-9238
471008,a1Y8c00000CmGvvEAF,0018c00002bI3ErAAK,a1W8c0000083rYcEAI,Vida Security,0018c00002axQfEAAU,Emaresa Ingenieros Y Representaciones S.A.,Emaresa Ingenieros Y Representaciones S.A.,Vida Security,Emaresa Ingenieros Y Representaciones S.A.,283047,83162400-0,EMP6141,A-9170


In [25]:
campaña_seleccionada.shape


(119231, 13)

In [26]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )


In [27]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )


In [28]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [29]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)
